The file comprises of data belonging to online bloggers in 'Twitch' regarding gaming and Twitch has been home to gaming streamers since its launch. The data contains top 1000 streamers from recent years. Data consists of number of columns like number of viewers, Number of active users, followers gained and other relevant coloumns 

In [1]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras


np.random.seed(1)
tf.random.set_seed(1)

In [2]:
df = pd.read_csv('./twitchdata.csv')
df.head(5)

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Follower Count,Followers gained,Views gained,Partnered,Mature,Language
0,Tfue,3671000070,123660,285644,29602,8938903,1,2068424,78998587,True,False,English
1,shroud,888505170,30240,471281,29612,7744066,1,833587,30621257,True,False,English
2,Myth,1479214575,134760,122552,9396,6726893,1,1421811,37384058,True,False,English
3,Rubius,2588632635,58275,240096,42948,5751354,1,3820532,58599449,True,False,Spanish
4,pokimane,964334055,56505,112160,16026,5367605,1,2085831,45579002,True,False,English


In [3]:
from sklearn import preprocessing

In [4]:
X_train = pd.read_csv("streamers_train_X.csv")
X_test = pd.read_csv("streamers_test_X.csv")
y_train = pd.read_csv("streamers_train_y.csv")
y_test = pd.read_csv("streamers_test_y.csv")

In [5]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

Default Logistic Model

In [6]:
log_reg_model = LogisticRegression(penalty='none', max_iter=900)
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [7]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548


In [8]:
#random search in log
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l2'],
    'solver': ['liblinear']
    
}

logreg = LogisticRegression()

rand_search_log = RandomizedSearchCV(estimator = logreg, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_log.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_log.best_score_}")
print(f"... with parameters: {rand_search_log.best_params_}")

bestRecallTree = rand_search_log.best_estimator_

C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=500. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best accuracy score is 0.9842857142857142
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}


C:\Users\davul\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
#svm with linear kernel
svm_lin_model = SVC(kernel="linear",probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [10]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"lin svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703


In [11]:
#svm with rbf kernel
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale',probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [12]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rbf svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776


In [13]:
svm_poly_model = SVC(kernel="poly",probability=True, degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [14]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance


,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761


In [15]:
#random search in SVM
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train,y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=500. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best accuracy score is 0.93
... with parameters: {'kernel': 'poly', 'gamma': 1, 'C': 1}


C:\Users\davul\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
model_preds = rand_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170


In [17]:
#grid search in SVM
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best accuracy score is 0.93
... with parameters: {'C': 1, 'gamma': 1, 'kernel': 'poly'}


C:\Users\davul\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
model_preds = grid_search.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170


In [19]:
#random with tree
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,42),  
    'min_samples_leaf': np.arange(1,42),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(1,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search_tree = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_tree.best_score_}")
print(f"... with parameters: {rand_search_tree.best_params_}")

bestRecallTree = rand_search_tree.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.9971428571428571
... with parameters: {'min_samples_split': 30, 'min_samples_leaf': 5, 'min_impurity_decrease': 0.0031, 'max_leaf_nodes': 34, 'max_depth': 32, 'criterion': 'gini'}


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [20]:
model_preds = rand_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170
0,rand_poly_tree,1.000000,1.000000,1.000000,1.000000


In [21]:
#grid with tree
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search_tree = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_tree.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search_tree.best_score_}")
print(f"... with parameters: {grid_search_tree.best_params_}")

bestRecallTree = grid_search_tree.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best accuracy score is 0.9971428571428571
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 36, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 3, 'min_samples_split': 3}


In [22]:
model_preds = grid_search_tree.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170
0,rand_poly_tree,1.000000,1.000000,1.000000,1.000000
0,grid_poly_tree,1.000000,1.000000,1.000000,1.000000


Neural Network

In [23]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(100,), solver='adam', max_iter=800)
_ = ann.fit(X_train, y_train)

C:\Users\davul\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 2.33 s


In [24]:
model_preds = ann.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Neural Networks-100", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170
0,rand_poly_tree,1.000000,1.000000,1.000000,1.000000
0,grid_poly_tree,1.000000,1.000000,1.000000,1.000000
0,Neural Networks-100,0.990000,1.000000,0.980645,0.990228


Deep  Networks

In [44]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [51]:
%%time

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(11))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(400, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) # final layer, 10 categories


# compile
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


Wall time: 72.9 ms


In [54]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 100)               1200      
                                                                 
 dense_29 (Dense)            (None, 100)               10100     
                                                                 
 dense_30 (Dense)            (None, 400)               40400     
                                                                 
 dense_31 (Dense)            (None, 10)                4010      
                                                                 
Total params: 55,710
Trainable params: 55,710
Non-trainable params: 0
_________________________________________________________________


In [53]:
%%time

# fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)


Epoch 1/20
7/7 [==============================] - 1s 76ms/step - loss: 1.9000 - accuracy: 0.4700 - val_loss: 1.4125 - val_accuracy: 0.7200
Epoch 2/20
7/7 [==============================] - 0s 10ms/step - loss: 1.0696 - accuracy: 0.7643 - val_loss: 0.7168 - val_accuracy: 0.7467
Epoch 3/20
7/7 [==============================] - 0s 10ms/step - loss: 0.5835 - accuracy: 0.7571 - val_loss: 0.4980 - val_accuracy: 0.7900
Epoch 4/20
7/7 [==============================] - 0s 9ms/step - loss: 0.4396 - accuracy: 0.8000 - val_loss: 0.4457 - val_accuracy: 0.7800
Epoch 5/20
7/7 [==============================] - 0s 9ms/step - loss: 0.3696 - accuracy: 0.8271 - val_loss: 0.3842 - val_accuracy: 0.8167
Epoch 6/20
7/7 [==============================] - 0s 10ms/step - loss: 0.3140 - accuracy: 0.8629 - val_loss: 0.3302 - val_accuracy: 0.8300
Epoch 7/20
7/7 [==============================] - 0s 9ms/step - loss: 0.2641 - accuracy: 0.8857 - val_loss: 0.2735 - val_accuracy: 0.8467
Epoch 8/20
7/7 [==============

In [55]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.10509748011827469, 0.9666666388511658]

In [56]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.11
accuracy: 96.67%


Wide and Deep Network

In [57]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=11))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(400, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [58]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [59]:
# Fit the model

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=20, batch_size=100)

Epoch 1/20
7/7 [==============================] - 1s 33ms/step - loss: 0.8931 - accuracy: 0.6486 - val_loss: 0.5095 - val_accuracy: 0.7800
Epoch 2/20
7/7 [==============================] - 0s 8ms/step - loss: 0.4144 - accuracy: 0.8286 - val_loss: 0.3239 - val_accuracy: 0.8733
Epoch 3/20
7/7 [==============================] - 0s 8ms/step - loss: 0.2852 - accuracy: 0.8700 - val_loss: 0.2197 - val_accuracy: 0.9033
Epoch 4/20
7/7 [==============================] - 0s 9ms/step - loss: 0.1974 - accuracy: 0.9257 - val_loss: 0.2167 - val_accuracy: 0.9067
Epoch 5/20
7/7 [==============================] - 0s 8ms/step - loss: 0.1553 - accuracy: 0.9343 - val_loss: 0.1611 - val_accuracy: 0.9333
Epoch 6/20
7/7 [==============================] - 0s 9ms/step - loss: 0.1162 - accuracy: 0.9543 - val_loss: 0.1553 - val_accuracy: 0.9167
Epoch 7/20
7/7 [==============================] - 0s 8ms/step - loss: 0.0872 - accuracy: 0.9629 - val_loss: 0.0940 - val_accuracy: 0.9600
Epoch 8/20
7/7 [=================

In [60]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.07626188546419144, 0.9866666793823242]

In [61]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.08
accuracy: 98.67%


Glot Uniform

In [66]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=11)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotUniform(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(10, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann


Wall time: 0 ns


In [67]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=64,
    dropout = 0.0
)


In [68]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':['adam','sgd'],
   
    
}
keras_clf.get_params().keys()



dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [69]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


3/3 [==============================] - 0s 2ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 1ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 2ms/step


...

C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 2ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 2ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan 0.79285714        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]
  warnings.warn(


In [70]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.0005,
 'optimizer': 'adam',
 'model__hidden_layer_sizes': (100, 90),
 'model__dropout': 0,
 'epochs': 100,
 'batch_size': 60}

In [71]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.0005, 'optimizer': 'adam', 'model__hidden_layer_sizes': (100, 90), 'model__dropout': 0, 'epochs': 100, 'batch_size': 60}


In [72]:
model_preds = ann.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"GlotUniform", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170
0,rand_poly_tree,1.000000,1.000000,1.000000,1.000000
0,grid_poly_tree,1.000000,1.000000,1.000000,1.000000
0,Neural Networks-100,0.990000,1.000000,0.980645,0.990228
0,GlotUniform,0.990000,1.000000,0.980645,0.990228


RandomGridSearch Normal

In [73]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = "accuracy"
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=11)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotNormal(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(10, activation='softmax'))
    ann.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return ann


Wall time: 0 ns


In [74]:
keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=64,
    dropout = 0.0
)


In [75]:
params = {
    'optimizer__learning_rate': [0.0005, 0.001, 0.005],
    'model__hidden_layer_sizes': [(70,),(90, ), (100,), (100, 90)],
    'model__dropout': [0, 0.1],
    'batch_size':[20, 60, 100],
    'epochs':[10, 50, 100],
    'optimizer':['adam','sgd'],
   
    
}
keras_clf.get_params().keys()



dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [76]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


3/3 [==============================] - 0s 1ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 1ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 1ms/step


...

C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 1ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\davul\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "C:\Users\davul\anaconda3\lib\site-packages\scikeras\wrappers.py", line 1054, in predict
    y_pred = self.target_encoder_.inverse_transform(y_pred)
  File

3/3 [==============================] - 0s 1ms/step


C:\Users\davul\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [77]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.001,
 'optimizer': 'sgd',
 'model__hidden_layer_sizes': (100, 90),
 'model__dropout': 0.1,
 'epochs': 10,
 'batch_size': 60}

In [78]:
model_preds = ann.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"RandomGrid Search Normal", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170
0,rand_poly_tree,1.000000,1.000000,1.000000,1.000000
0,grid_poly_tree,1.000000,1.000000,1.000000,1.000000
0,Neural Networks-100,0.990000,1.000000,0.980645,0.990228
0,GlotUniform,0.990000,1.000000,0.980645,0.990228


In [79]:
performance.sort_values(by='Accuracy')

,model,Accuracy,Precision,Recall,F1
0,rand_poly_svm,0.926667,0.978417,0.877419,0.925170
0,grid_poly_svm,0.926667,0.978417,0.877419,0.925170
0,rbf svm,0.940000,0.992806,0.890323,0.938776
0,poly svm,0.943333,0.985915,0.903226,0.942761
0,lin svm,0.953333,1.000000,0.909677,0.952703
0,Neural Networks-100,0.990000,1.000000,0.980645,0.990228
0,GlotUniform,0.990000,1.000000,0.980645,0.990228
0,RandomGrid Search Normal,0.990000,1.000000,0.980645,0.990228
0,default logistic,0.993333,0.993548,0.993548,0.993548
0,rand_poly_tree,1.000000,1.000000,1.000000,1.000000


From the above analysis we can see that randomised poly tree and gridsearch poly tree models are over fitted and as compared with  Neural networks and GlotNormal and GlotUniform Default logistic model is performing well when sorted with metric as Accuracy.